# Step 9: Mini GPT 구현 - 나만의 언어 모델 만들기

이제 지금까지 배운 모든 것을 종합하여 작은 GPT 모델을 처음부터 구현해봅시다!

## 학습 목표
1. GPT 아키텍처 이해 (Decoder-only Transformer)
2. 텍스트 토큰화와 데이터 준비
3. 모델 학습 파이프라인 구축
4. 텍스트 생성 전략 구현
5. 모델 평가와 분석

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import math
import os
import json
from collections import Counter
import re

# 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 재현성
torch.manual_seed(42)
np.random.seed(42)

## 1. 토큰화 (Tokenization)

텍스트를 모델이 이해할 수 있는 토큰으로 변환합니다.

In [ ]:
class SimpleTokenizer:
    """간단한 문자 수준 토크나이저"""
    def __init__(self):
        self.char_to_idx = {}
        self.idx_to_char = {}
        self.vocab_size = 0
        
    def fit(self, texts):
        """텍스트에서 어휘 구축"""
        # 모든 고유 문자 수집
        chars = set()
        for text in texts:
            chars.update(text)
        
        # 특수 토큰 추가
        chars = ['<PAD>', '<SOS>', '<EOS>', '<UNK>'] + sorted(list(chars))
        
        # 인덱스 매핑 생성
        self.char_to_idx = {char: idx for idx, char in enumerate(chars)}
        self.idx_to_char = {idx: char for idx, char in enumerate(chars)}
        self.vocab_size = len(chars)
        
        print(f"어휘 크기: {self.vocab_size}")
        print(f"어휘 예시: {chars[:20]}...")
    
    def encode(self, text):
        """텍스트를 인덱스로 변환"""
        return [self.char_to_idx.get(char, self.char_to_idx['<UNK>']) for char in text]
    
    def decode(self, indices):
        """인덱스를 텍스트로 변환"""
        return ''.join([self.idx_to_char.get(idx, '<UNK>') for idx in indices])

# 더 고급 토크나이저: 단어 수준
class WordTokenizer:
    """단어 수준 토크나이저"""
    def __init__(self, vocab_size=None):
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.vocab_size = 0
        self.max_vocab_size = vocab_size
    
    def fit(self, texts):
        """텍스트에서 어휘 구축"""
        # 단어 빈도 계산
        word_counts = Counter()
        for text in texts:
            words = text.lower().split()
            word_counts.update(words)
        
        # 빈도순으로 정렬
        if self.max_vocab_size:
            most_common = word_counts.most_common(self.max_vocab_size - 4)
        else:
            most_common = word_counts.most_common()
        
        # 특수 토큰 추가
        vocab = ['<PAD>', '<SOS>', '<EOS>', '<UNK>'] + [word for word, _ in most_common]
        
        # 인덱스 매핑
        self.word_to_idx = {word: idx for idx, word in enumerate(vocab)}
        self.idx_to_word = {idx: word for idx, word in enumerate(vocab)}
        self.vocab_size = len(vocab)
        
        print(f"어휘 크기: {self.vocab_size}")
        print(f"가장 빈번한 단어: {[word for word, _ in most_common[:10]]}")
    
    def encode(self, text):
        """텍스트를 인덱스로 변환"""
        words = text.lower().split()
        return [self.word_to_idx.get(word, self.word_to_idx['<UNK>']) for word in words]
    
    def decode(self, indices):
        """인덱스를 텍스트로 변환"""
        words = [self.idx_to_word.get(idx, '<UNK>') for idx in indices]
        return ' '.join(words)

# 예시 텍스트
sample_texts = [
    "Deep learning is a subset of machine learning.",
    "Machine learning is a subset of artificial intelligence.",
    "Neural networks are inspired by the human brain.",
    "Transformers have revolutionized natural language processing."
]

# 토크나이저 테스트
char_tokenizer = SimpleTokenizer()
char_tokenizer.fit(sample_texts)

# 인코딩/디코딩 예시
test_text = "Deep learning"
encoded = char_tokenizer.encode(test_text)
decoded = char_tokenizer.decode(encoded)

print(f"\n원본 텍스트: '{test_text}'")
print(f"인코딩: {encoded}")
print(f"디코딩: '{decoded}'")

## 2. 데이터셋 준비

In [ ]:
class GPTDataset(Dataset):
    """GPT 학습을 위한 데이터셋"""
    def __init__(self, texts, tokenizer, seq_length=128):
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        
        # 모든 텍스트를 하나로 합치고 토큰화
        all_text = ' '.join(texts)
        self.tokens = tokenizer.encode(all_text)
        
        print(f"총 토큰 수: {len(self.tokens)}")
        print(f"시퀀스 길이: {seq_length}")
        print(f"가능한 시퀀스 수: {len(self.tokens) - seq_length}")
    
    def __len__(self):
        return len(self.tokens) - self.seq_length
    
    def __getitem__(self, idx):
        # 입력: [idx:idx+seq_length]
        # 타겟: [idx+1:idx+seq_length+1]
        input_seq = torch.tensor(self.tokens[idx:idx+self.seq_length], dtype=torch.long)
        target_seq = torch.tensor(self.tokens[idx+1:idx+self.seq_length+1], dtype=torch.long)
        
        return input_seq, target_seq

# 더 큰 텍스트 데이터 생성 (셰익스피어 스타일)
shakespeare_style = """
To be or not to be that is the question.
Whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune.
Or to take arms against a sea of troubles and by opposing end them.
To die to sleep no more and by a sleep to say we end.
The heartache and the thousand natural shocks that flesh is heir to.
Tis a consummation devoutly to be wished to die to sleep.
To sleep perchance to dream ay theres the rub.
For in that sleep of death what dreams may come.
"""

# 데이터셋 생성
texts = [shakespeare_style] * 10  # 더 많은 데이터를 위해 반복
dataset = GPTDataset(texts, char_tokenizer, seq_length=64)

# 샘플 확인
input_seq, target_seq = dataset[0]
print(f"\n입력 시퀀스: {input_seq.shape}")
print(f"타겟 시퀀스: {target_seq.shape}")
print(f"\n입력 텍스트: '{char_tokenizer.decode(input_seq.tolist()[:20])}...'")
print(f"타겟 텍스트: '{char_tokenizer.decode(target_seq.tolist()[:20])}...'")

## 3. Mini GPT 모델 구현

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.size()
        
        # Q, K, V 계산
        Q = self.W_q(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        
        # Attention 계산
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        context = torch.matmul(attn_weights, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        
        output = self.W_o(context)
        
        return output, attn_weights

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.gelu = nn.GELU()
        
    def forward(self, x):
        return self.fc2(self.dropout(self.gelu(self.fc1(x))))

class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, n_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention with residual
        attn_output, attn_weights = self.attention(self.ln1(x), mask)
        x = x + self.dropout(attn_output)
        
        # Feed-forward with residual
        ff_output = self.feed_forward(self.ln2(x))
        x = x + self.dropout(ff_output)
        
        return x, attn_weights

class MiniGPT(nn.Module):
    def __init__(self, vocab_size, d_model=256, n_heads=8, n_layers=6, 
                 d_ff=1024, max_seq_len=512, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        # 토큰 임베딩
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        self.dropout = nn.Dropout(dropout)
        
        # Transformer 블록
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        
        # 최종 레이어
        self.ln_f = nn.LayerNorm(d_model)
        self.fc_out = nn.Linear(d_model, vocab_size)
        
        # 가중치 초기화
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, x, targets=None):
        batch_size, seq_len = x.size()
        
        # 위치 인덱스
        positions = torch.arange(0, seq_len).expand(batch_size, seq_len).to(x.device)
        
        # 임베딩
        tok_emb = self.token_embedding(x)
        pos_emb = self.position_embedding(positions)
        x = self.dropout(tok_emb + pos_emb)
        
        # Causal mask 생성
        mask = torch.tril(torch.ones(seq_len, seq_len)).view(1, 1, seq_len, seq_len).to(x.device)
        
        # Transformer 블록 통과
        attention_weights = []
        for block in self.blocks:
            x, attn_w = block(x, mask)
            attention_weights.append(attn_w)
        
        # 최종 출력
        x = self.ln_f(x)
        logits = self.fc_out(x)
        
        # 손실 계산 (학습 시)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        
        return logits, loss, attention_weights
    
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, do_sample=True):
        """텍스트 생성"""
        self.eval()
        for _ in range(max_new_tokens):
            # 컨텍스트 크기 제한
            idx_cond = idx if idx.size(1) <= self.max_seq_len else idx[:, -self.max_seq_len:]
            
            # 예측
            logits, _, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            
            # 샘플링
            if do_sample:
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)
            
            # 다음 토큰 추가
            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx

# 모델 생성
model = MiniGPT(
    vocab_size=char_tokenizer.vocab_size,
    d_model=128,
    n_heads=4,
    n_layers=4,
    d_ff=512,
    max_seq_len=256,
    dropout=0.1
).to(device)

# 모델 정보
total_params = sum(p.numel() for p in model.parameters())
print(f"Mini GPT 모델 생성 완료!")
print(f"총 파라미터 수: {total_params:,}")
print(f"모델 크기: {total_params * 4 / 1024 / 1024:.2f} MB (float32 기준)")

## 4. 모델 학습

In [ ]:
def train_model(model, dataset, epochs=10, batch_size=32, learning_rate=3e-4):
    """모델 학습"""
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    
    # 학습률 스케줄러 (선택사항)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    model.train()
    losses = []
    
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{epochs}')
        
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass
            logits, loss, _ = model(inputs, targets)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})
        
        avg_loss = total_loss / len(dataloader)
        losses.append(avg_loss)
        scheduler.step()
        
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")
        
        # 샘플 생성 (매 에폭마다)
        if (epoch + 1) % 2 == 0:
            generate_sample(model, char_tokenizer, "To be or ")
    
    return losses

def generate_sample(model, tokenizer, prompt, max_length=100, temperature=1.0):
    """샘플 텍스트 생성"""
    model.eval()
    
    # 프롬프트 인코딩
    tokens = tokenizer.encode(prompt)
    x = torch.tensor(tokens).unsqueeze(0).to(device)
    
    # 생성
    with torch.no_grad():
        generated = model.generate(x, max_length, temperature)
    
    # 디코딩
    generated_text = tokenizer.decode(generated[0].tolist())
    print(f"\n생성된 텍스트: '{generated_text}'\n")
    
    model.train()

# 학습 실행
print("학습 시작...")
losses = train_model(model, dataset, epochs=20, batch_size=16, learning_rate=1e-3)

## 5. 학습 결과 분석

In [ ]:
# 학습 곡선 시각화
plt.figure(figsize=(10, 6))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

# 다양한 온도로 텍스트 생성
def generate_with_different_temperatures(model, tokenizer, prompt):
    temperatures = [0.5, 0.8, 1.0, 1.2, 1.5]
    
    for temp in temperatures:
        print(f"\n--- Temperature: {temp} ---")
        model.eval()
        
        tokens = tokenizer.encode(prompt)
        x = torch.tensor(tokens).unsqueeze(0).to(device)
        
        with torch.no_grad():
            generated = model.generate(x, 100, temperature=temp)
        
        generated_text = tokenizer.decode(generated[0].tolist())
        print(generated_text)

print("다양한 온도로 텍스트 생성:")
generate_with_different_temperatures(model, char_tokenizer, "To be or ")

## 6. Attention 패턴 시각화

In [ ]:
def visualize_attention_patterns(model, tokenizer, text):
    """Attention 패턴 시각화"""
    model.eval()
    
    # 텍스트 인코딩
    tokens = tokenizer.encode(text)
    x = torch.tensor(tokens).unsqueeze(0).to(device)
    
    # Forward pass
    with torch.no_grad():
        logits, _, attention_weights = model(x)
    
    # 첫 번째 레이어, 첫 번째 헤드의 attention 시각화
    attn = attention_weights[0][0, 0].cpu().numpy()  # (seq_len, seq_len)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(attn[:len(text), :len(text)], 
                xticklabels=list(text),
                yticklabels=list(text),
                cmap='Blues',
                cbar_kws={'label': 'Attention Weight'})
    plt.title('Attention Pattern (Layer 1, Head 1)')
    plt.xlabel('Keys')
    plt.ylabel('Queries')
    plt.tight_layout()
    plt.show()
    
    # 여러 레이어의 평균 attention
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    for i in range(min(4, len(attention_weights))):
        avg_attn = attention_weights[i][0].mean(dim=0).cpu().numpy()
        
        im = axes[i].imshow(avg_attn[:len(text), :len(text)], 
                           cmap='Blues', aspect='auto')
        axes[i].set_title(f'Layer {i+1} (Average across heads)')
        axes[i].set_xlabel('Position')
        axes[i].set_ylabel('Position')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()

# Attention 패턴 시각화
test_text = "To be or not to be"
print(f"텍스트: '{test_text}'")
visualize_attention_patterns(model, char_tokenizer, test_text)

## 7. 생성 전략 비교

In [ ]:
class AdvancedGenerator:
    """고급 텍스트 생성 전략"""
    
    @staticmethod
    def greedy_search(model, tokenizer, prompt, max_length=100):
        """Greedy 디코딩"""
        model.eval()
        tokens = tokenizer.encode(prompt)
        x = torch.tensor(tokens).unsqueeze(0).to(device)
        
        with torch.no_grad():
            generated = model.generate(x, max_length, temperature=1.0, do_sample=False)
        
        return tokenizer.decode(generated[0].tolist())
    
    @staticmethod
    def top_k_sampling(model, tokenizer, prompt, max_length=100, k=10, temperature=1.0):
        """Top-k 샘플링"""
        model.eval()
        tokens = tokenizer.encode(prompt)
        x = torch.tensor(tokens).unsqueeze(0).to(device)
        
        for _ in range(max_length):
            with torch.no_grad():
                logits, _, _ = model(x)
                logits = logits[:, -1, :] / temperature
            
            # Top-k 필터링
            top_k_values, top_k_indices = torch.topk(logits, k, dim=-1)
            probs = F.softmax(top_k_values, dim=-1)
            
            # 샘플링
            idx = torch.multinomial(probs, num_samples=1)
            next_token = top_k_indices.gather(-1, idx)
            
            x = torch.cat((x, next_token), dim=1)
        
        return tokenizer.decode(x[0].tolist())
    
    @staticmethod
    def top_p_sampling(model, tokenizer, prompt, max_length=100, p=0.9, temperature=1.0):
        """Top-p (nucleus) 샘플링"""
        model.eval()
        tokens = tokenizer.encode(prompt)
        x = torch.tensor(tokens).unsqueeze(0).to(device)
        
        for _ in range(max_length):
            with torch.no_grad():
                logits, _, _ = model(x)
                logits = logits[:, -1, :] / temperature
            
            # 확률 계산 및 정렬
            probs = F.softmax(logits, dim=-1)
            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            
            # 누적 확률 계산
            cumsum_probs = torch.cumsum(sorted_probs, dim=-1)
            
            # Top-p 마스크
            mask = cumsum_probs > p
            mask[..., 1:] = mask[..., :-1].clone()
            mask[..., 0] = False
            
            # 필터링된 확률에서 샘플링
            sorted_probs[mask] = 0
            sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
            
            idx = torch.multinomial(sorted_probs, num_samples=1)
            next_token = sorted_indices.gather(-1, idx)
            
            x = torch.cat((x, next_token), dim=1)
        
        return tokenizer.decode(x[0].tolist())

# 다양한 생성 전략 비교
prompt = "To be or "
generator = AdvancedGenerator()

print("=== 다양한 생성 전략 비교 ===")
print(f"프롬프트: '{prompt}'\n")

print("1. Greedy Search:")
print(generator.greedy_search(model, char_tokenizer, prompt, 50))

print("\n2. Top-k Sampling (k=5):")
print(generator.top_k_sampling(model, char_tokenizer, prompt, 50, k=5))

print("\n3. Top-p Sampling (p=0.9):")
print(generator.top_p_sampling(model, char_tokenizer, prompt, 50, p=0.9))

## 8. 모델 저장과 불러오기

In [ ]:
# 모델 저장
def save_model(model, tokenizer, path='mini_gpt_model'):
    """모델과 토크나이저 저장"""
    # 디렉토리 생성
    os.makedirs(path, exist_ok=True)
    
    # 모델 저장
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'vocab_size': model.token_embedding.num_embeddings,
            'd_model': model.d_model,
            'n_heads': model.blocks[0].attention.n_heads,
            'n_layers': len(model.blocks),
            'd_ff': model.blocks[0].feed_forward.fc1.out_features,
            'max_seq_len': model.max_seq_len,
        }
    }, os.path.join(path, 'model.pt'))
    
    # 토크나이저 저장
    tokenizer_data = {
        'char_to_idx': tokenizer.char_to_idx,
        'idx_to_char': tokenizer.idx_to_char,
        'vocab_size': tokenizer.vocab_size
    }
    with open(os.path.join(path, 'tokenizer.json'), 'w') as f:
        json.dump(tokenizer_data, f)
    
    print(f"모델이 '{path}' 디렉토리에 저장되었습니다.")

# 모델 불러오기
def load_model(path='mini_gpt_model'):
    """저장된 모델과 토크나이저 불러오기"""
    # 체크포인트 불러오기
    checkpoint = torch.load(os.path.join(path, 'model.pt'), map_location=device)
    
    # 모델 재생성
    config = checkpoint['model_config']
    model = MiniGPT(
        vocab_size=config['vocab_size'],
        d_model=config['d_model'],
        n_heads=config['n_heads'],
        n_layers=config['n_layers'],
        d_ff=config['d_ff'],
        max_seq_len=config['max_seq_len']
    ).to(device)
    
    # 가중치 로드
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 토크나이저 로드
    with open(os.path.join(path, 'tokenizer.json'), 'r') as f:
        tokenizer_data = json.load(f)
    
    tokenizer = SimpleTokenizer()
    tokenizer.char_to_idx = tokenizer_data['char_to_idx']
    tokenizer.idx_to_char = {int(k): v for k, v in tokenizer_data['idx_to_char'].items()}
    tokenizer.vocab_size = tokenizer_data['vocab_size']
    
    print(f"모델이 '{path}' 디렉토리에서 로드되었습니다.")
    return model, tokenizer

# 모델 저장
save_model(model, char_tokenizer)

# 모델 불러오기 테스트
loaded_model, loaded_tokenizer = load_model()
print("\n불러온 모델로 텍스트 생성:")
generate_sample(loaded_model, loaded_tokenizer, "To die ", max_length=50)

## 9. 성능 분석 및 개선 방안

In [ ]:
def analyze_model_performance(model, dataset, tokenizer):
    """모델 성능 분석"""
    model.eval()
    
    # Perplexity 계산
    total_loss = 0
    total_tokens = 0
    
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Calculating perplexity"):
            inputs, targets = inputs.to(device), targets.to(device)
            _, loss, _ = model(inputs, targets)
            
            total_loss += loss.item() * inputs.size(0) * inputs.size(1)
            total_tokens += inputs.size(0) * inputs.size(1)
    
    perplexity = math.exp(total_loss / total_tokens)
    print(f"\nPerplexity: {perplexity:.2f}")
    
    # 생성 품질 평가
    print("\n=== 생성 품질 평가 ===")
    
    test_prompts = [
        "To be or ",
        "Whether tis ",
        "The heart",
        "To sleep "
    ]
    
    for prompt in test_prompts:
        print(f"\n프롬프트: '{prompt}'")
        tokens = tokenizer.encode(prompt)
        x = torch.tensor(tokens).unsqueeze(0).to(device)
        
        with torch.no_grad():
            generated = model.generate(x, 30, temperature=0.8)
        
        generated_text = tokenizer.decode(generated[0].tolist())
        print(f"생성: '{generated_text}'")
    
    # 모델 통계
    print("\n=== 모델 통계 ===")
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"총 파라미터 수: {total_params:,}")
    print(f"학습 가능한 파라미터 수: {trainable_params:,}")
    print(f"모델 크기: {total_params * 4 / 1024 / 1024:.2f} MB")
    
    # 레이어별 파라미터 분포
    layer_params = {}
    for name, param in model.named_parameters():
        layer_type = name.split('.')[0]
        if layer_type not in layer_params:
            layer_params[layer_type] = 0
        layer_params[layer_type] += param.numel()
    
    print("\n레이어별 파라미터 분포:")
    for layer, count in layer_params.items():
        print(f"  {layer}: {count:,} ({count/total_params*100:.1f}%)")

# 성능 분석 실행
analyze_model_performance(model, dataset, char_tokenizer)

## 10. 개선 방안 및 실험

In [ ]:
# 개선 방안 시각화
def visualize_improvements():
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. 모델 크기 vs 성능
    model_sizes = [1e6, 10e6, 100e6, 1e9, 10e9, 100e9]  # 파라미터 수
    performance = [50, 35, 25, 15, 10, 7]  # Perplexity (낮을수록 좋음)
    
    axes[0, 0].loglog(model_sizes, performance, 'b-o', linewidth=2, markersize=8)
    axes[0, 0].axvline(x=total_params, color='r', linestyle='--', label='Our Model')
    axes[0, 0].set_xlabel('Number of Parameters')
    axes[0, 0].set_ylabel('Perplexity')
    axes[0, 0].set_title('Model Size vs Performance')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 학습 데이터 크기의 영향
    data_sizes = [1e3, 1e4, 1e5, 1e6, 1e7, 1e8]  # 토큰 수
    performance_data = [80, 50, 30, 20, 15, 12]
    
    axes[0, 1].loglog(data_sizes, performance_data, 'g-o', linewidth=2, markersize=8)
    axes[0, 1].axvline(x=len(dataset.tokens), color='r', linestyle='--', label='Our Data')
    axes[0, 1].set_xlabel('Training Tokens')
    axes[0, 1].set_ylabel('Perplexity')
    axes[0, 1].set_title('Data Size vs Performance')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 다양한 아키텍처 비교
    architectures = ['MLP', 'RNN', 'LSTM', 'Transformer', 'GPT-2', 'GPT-3']
    params = [1e6, 10e6, 20e6, 100e6, 1.5e9, 175e9]
    perplexities = [150, 80, 60, 30, 20, 10]
    
    axes[1, 0].scatter(params, perplexities, s=100, alpha=0.6)
    for i, arch in enumerate(architectures):
        axes[1, 0].annotate(arch, (params[i], perplexities[i]), 
                           xytext=(5, 5), textcoords='offset points')
    axes[1, 0].set_xscale('log')
    axes[1, 0].set_xlabel('Parameters')
    axes[1, 0].set_ylabel('Perplexity')
    axes[1, 0].set_title('Architecture Comparison')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 개선 방안
    improvements = [
        'Larger Model',
        'More Data',
        'Better Tokenization',
        'Longer Training',
        'Advanced Techniques'
    ]
    expected_gains = [30, 40, 15, 10, 25]  # 예상 개선율 (%)
    
    axes[1, 1].barh(improvements, expected_gains, color='skyblue')
    axes[1, 1].set_xlabel('Expected Improvement (%)')
    axes[1, 1].set_title('Potential Improvements')
    axes[1, 1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

visualize_improvements()

print("\n=== 개선 방안 ===")
print("1. **모델 크기 증가**:")
print("   - 더 많은 레이어와 헤드 추가")
print("   - Hidden dimension 증가")
print("   - 현재: ~500K 파라미터 → 목표: 10M+ 파라미터")
print("\n2. **데이터 증가**:")
print("   - 더 큰 텍스트 코퍼스 사용")
print("   - 다양한 장르와 스타일 포함")
print("   - 데이터 증강 기법 적용")
print("\n3. **토크나이저 개선**:")
print("   - BPE (Byte-Pair Encoding) 사용")
print("   - SentencePiece 등 고급 토크나이저")
print("   - 더 효율적인 어휘 구성")
print("\n4. **학습 기법 개선**:")
print("   - Mixed precision training")
print("   - Gradient accumulation")
print("   - Learning rate scheduling")
print("   - Warm-up steps")
print("\n5. **고급 기법**:")
print("   - Flash Attention")
print("   - Rotary Position Embedding (RoPE)")
print("   - Layer-wise learning rate")
print("   - Knowledge distillation")

## 11. 연습 문제

In [ ]:
# 문제 1: Beam Search 구현
def beam_search(model, tokenizer, prompt, beam_width=3, max_length=50):
    """
    Beam search를 사용한 텍스트 생성
    
    힌트:
    - 각 단계에서 beam_width개의 가장 확률 높은 시퀀스 유지
    - 각 시퀀스의 로그 확률 합계 추적
    - 최종적으로 가장 높은 점수의 시퀀스 반환
    """
    # TODO: 구현하기
    pass

# 문제 2: 조건부 생성 구현
class ConditionalGPT(MiniGPT):
    """
    조건부 텍스트 생성을 위한 GPT
    예: 감정, 스타일, 주제 등을 조건으로 제공
    
    힌트:
    - 조건 임베딩 추가
    - 조건과 텍스트 임베딩 결합
    """
    def __init__(self, vocab_size, num_conditions, *args, **kwargs):
        super().__init__(vocab_size, *args, **kwargs)
        # TODO: 조건 임베딩 레이어 추가
        pass

# 문제 3: 효율적인 Attention 구현
class EfficientAttention(nn.Module):
    """
    메모리 효율적인 Attention 구현
    예: Sparse Attention, Local Attention 등
    
    힌트:
    - 모든 위치가 아닌 일부만 attend
    - 또는 청크 단위로 attention 계산
    """
    def __init__(self, d_model, n_heads, window_size=128):
        super().__init__()
        # TODO: 구현하기
        pass

## 정리

이번 튜토리얼에서 배운 내용:
1. **GPT 아키텍처**: Decoder-only Transformer
2. **토큰화**: 텍스트를 모델이 이해할 수 있는 형태로 변환
3. **학습 파이프라인**: 데이터 준비부터 모델 학습까지
4. **텍스트 생성**: 다양한 샘플링 전략
5. **성능 분석**: Perplexity와 생성 품질 평가

### Mini GPT의 핵심 구성 요소:
- **Self-Attention**: 이전 토큰들의 정보 활용
- **Causal Masking**: 미래 정보 차단
- **Position Embedding**: 순서 정보 인코딩
- **Autoregressive Generation**: 한 토큰씩 순차적 생성

### 실제 GPT와의 차이:
- **모델 크기**: 실제 GPT는 수십억~수천억 파라미터
- **데이터**: 인터넷 규모의 텍스트로 학습
- **토크나이저**: BPE 등 더 효율적인 방법 사용
- **최적화**: 분산 학습, mixed precision 등

다음 단계에서는 이러한 대규모 언어 모델을 실제로 사용하는 방법을 배워보겠습니다!